In [0]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt

In [0]:
# Carregar conjunto de dados de exemplo
dados = fetch_california_housing()
X = dados.data
y = dados.target

In [0]:
df_xy = pd.DataFrame(X, columns=dados.feature_names)
df_xy[dados.target_names[0]] = y

In [0]:
df_xy.head()

In [0]:
# Histogram of each feature
df_xy.hist(figsize=(12, 10), bins=30)
plt.tight_layout()
plt.show()

# Correlation heatmap
plt.figure(figsize=(10, 8))
corr = df_xy.corr()
plt.imshow(corr, cmap='coolwarm', interpolation='none')
plt.colorbar()
plt.xticks(range(len(corr)), corr.columns, rotation=90)
plt.yticks(range(len(corr)), corr.columns)
plt.title("Correlation Heatmap")
plt.show()

# Scatter plots for each feature vs target
for col in dados.feature_names:
    plt.figure(figsize=(6, 4))
    plt.scatter(df_xy[col], df_xy[dados.target_names[0]], alpha=0.3)
    plt.xlabel(col)
    plt.ylabel(dados.target_names[0])
    plt.title(f"{col} vs {dados.target_names[0]}")
    plt.show()

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [0]:
X_mean = X_train.mean(axis=0)
X_std  = X_train.std(axis=0)

X_train_s = (X_train - X_mean) / X_std
X_test_s  = (X_test - X_mean) / X_std

In [0]:
learning_rate=[0.0001,0.001,0.01,0.1,0.5]

In [0]:
result={}
lista=[]

In [0]:
for i in learning_rate:
    #Desenvolvendo o gradiente descendente
    #Quantas vezes o gradiente descendente vai rodar
    epochs=10000
    #Variavel para armazenar a evolução do erro
    loss_history=[]
    #Criamos uma matriz de zeros que será atribuida aos pesos de cada variavel
    W=np.zeros(X_train_s.shape[1])
    #Variavel para armazenar o intercepto
    b=0.0
    loss_old=0
    for epoch in range(epochs):
        #Aqui que ocorre a previsão
        #multiplicamos os dados de x_train_S pela matriz w e somamos o intercepto
        #isso nada mais e que a formula da regressao linear
        #ŷ = Wx + b
        yhat=X_train_s.dot(W)+b
        #Aqui observamos a diferença do que prevemos para o real da base de treino
        # Esse é o erro da previsão: o quanto o modelo errou para cada ponto
        residuo=yhat-y_train
        #esse é o erro quadrático médio
        # Erro quadrático médio = quanto estamos errando em média
        loss=(residuo**2).mean()
        #Guardamos ele na variavel loss_history
        loss_history.append(loss)
        #Aqui calcula quanto devemos ajustar para cada peso
        #Isso é a derivada da função de custo MSE em relação aos pesos W. 
        #dividimos  2 pelo  tamanho da base de treino 
        #pegamos a matriz transposta de x_train_s e multiplicamos com a diferença entre o que prevemos e o real
        dW=(2/len(X_train_s))*X_train_s.T.dot(residuo)
        #Aqui calculamos o intercepto
        db=(2/len(X_train_s))*residuo.sum()
        #atualizamos os pesos
        W-=i*dW
        b-=i*db
        # Se o erro quase não muda mais, paramos, porque o modelo não tem mais nada relevante para aprender
        if abs(loss_old-loss)<1e-4:
            yhat_test=X_test_s.dot(W)+b
            residuo=yhat_test-y_test
            mse=(residuo**2).mean()
            rmse=np.sqrt((residuo**2).mean())
            r2= r2_score(y_test, yhat_test)
            plt.plot(loss_history,label=f"learning_rate:{i}")
            plt.xlabel("Epoch")
            plt.ylabel("Loss")
            plt.legend()
            result={'learning_rate':i,'epochs':epoch,'loss':loss,'r2':r2,'mse':mse,'rmse':rmse,'W':W,'b':b}
            lista.append(result)

            
            
            break
        
        #Atualizamos a variavel de loss_old para calcular o próximo epoch e fazer a comparação
        loss_old=loss


    
    



In [0]:
df=pd.DataFrame(lista)

In [0]:
df.W.head()

In [0]:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LinearRegression())
])

In [0]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

In [0]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2   = r2_score(y_test, y_pred)
print(f"1D | MSE={mse:.5f} | R2={r2:.5f} | RMSE={rmse:.5f}")

In [0]:
plt.scatter(y_test,y_pred,label="Predicted vs Real")
plt.plot([y_test.min(),y_test.max()],
         [y_test.min(),y_test.max()],
         color='red',
         label="Ideal Line",
         linestyle='--')
plt.xlabel("Real")
plt.ylabel("Predicted")
plt.legend()
plt.show()